## Objective

The goal of this notebook is to extract and clean SOV table from markdown-formatted chunks (derived from Excel tables), ensuring accurate mapping using a predefined header dictionary.


**Imports & Setup**

In [1]:
import time
import os
import pandas as pd
import re
import json
import utils
from typing import List, Dict, Union
import tiktoken 
from tqdm import tqdm
from IPython.display import display
import warnings
from openpyxl import load_workbook

import openai

**This function is designed to extract, clean, and standardize headers (column names) from an Excel file, and also extract important metadata like carrier, insured, broker, and SOV file name from the preview of the Excel sheet.**

# 1. Load OPENAI 

In [2]:
def get_api_key_details(model="GPT4o-mini", secret_file_path=r"gpt_api_details.csv"):
    """
    Returns the api key details for the given model: "GPT4o-mini" or "GPT4o"

    Returned as a dictionary with keys: 'Model', 'model_name', 'deployment_name', 'api_key', 'api_base', 'api_type', 'api_version'
    """
    df = pd.read_csv(secret_file_path)
    row_dicts = df.to_dict(orient='records')
    for row_dict in row_dicts:
        if row_dict['Model'] == model:
            return row_dict
    return None

In [3]:
api_details = get_api_key_details(model="GPT4.1", secret_file_path=r"gpt_api_details.csv")
# print(api_details)
model_key = api_details['api_key']
model_deployment_name = api_details['deployment_name']
model_version = api_details['api_version']
model_base = api_details['api_base']

In [4]:
gpt_client = openai.AzureOpenAI(
        azure_endpoint=model_base,
        api_key=model_key,
        azure_deployment=model_deployment_name,
        api_version=model_version,
    )

In [5]:
def get_completion(prompt: str, system_prompt: str, max_tokens:int=16384,active_model = None) -> str:
    """
    Get completion from OpenAI API
    Args:
        prompt (str): Prompt to generate completion for
    Returns:
        str: Generated completion
    """
 
    # gpt_client, model_details= switch_model(active_model_key=active_model)
    # logger.debug(f"Model Name: {model_details}")
    response = gpt_client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt,
                    }
                ],
            },
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        seed=0,
        top_p=0.01,
        max_tokens=max_tokens,
        response_format={"type": "json_object"}
    )
    input_tokens = response.usage.prompt_tokens #["usage"]["prompt_tokens"]
    output_tokens = response.usage.completion_tokens #["usage"]["completion_tokens"]
    input_cost = (input_tokens) * 0.00001
    output_cost = (output_tokens) * 0.0002
    total_cost = input_cost + output_cost
    tokens_used = response.usage.total_tokens
    return response.choices[0].message.content.strip()

# 2. Header Mapping Functions

### 2.1 Get Header Row Number (using LLM) : Working Fine

In [6]:
def extract_header_row_number_only(excel_path, sheet_name=None):
    # Determine file extension
    file_ext = os.path.splitext(excel_path)[1].lower()

    if sheet_name is None:
        sheet_name = pd.ExcelFile(excel_path).sheet_names[0] 

    # Use appropriate engine for .xls files
    if file_ext == ".xls":
        preview_df = pd.read_excel(excel_path, sheet_name=sheet_name, nrows=40, header=None, engine="xlrd")
    else:
        preview_df = pd.read_excel(excel_path, sheet_name=sheet_name, nrows=40, header=None)
    
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        # Clean all string cells: remove \n, \r, and extra spaces
        preview_df = preview_df.applymap(
            lambda x: re.sub(r'\s+', ' ', str(x).replace('\n', ' ').replace('\r', ' ')).strip()
            if isinstance(x, str) else x
        )
        
    markdown_chunk = preview_df.to_markdown(index=True)

    system_prompt = """
You are a helpful assistant that analyzes Excel sheet previews formatted as markdown tables.
Your only task is to identify which row number contains the column headers in the table.
Only focus on getting the header row number — do not extract or interpret data rows.
"""

    user_prompt = f"""
You are given a markdown-formatted table extracted from excel along with Standard Header Names & Variants:

### Standard Header Names & Variants
 
These are the acceptable variations you can match against (case-insensitive, ignore punctuation and spacing):
 
- **Location Name**: Location Name, Name, Description, *Location, *Location Name, *Description  
- **Location Number**: Location Number, Location #, Loc No, Location  
- **Bldg No**: *Bldg No., *Bldg No, Bldg No., Building Number, Building #, Bldg. #  
- **Property ID**: Property ID, PID, *Property ID  
- **House Number**: House number, Address number, AddressNum, Address  
- **Street Address**: Street Address, address, *Street Address  
- **City**: City, *City  
- **Zip Code**: Zip, Zip code, *Zip code, Pincode, Pin, *Zip  
- **State**: State, *State Code, State Code, ST  
- **County**: County  
- **Country**: Country, *Country  
- **Number of Units**: Number of Units, Nr. of risks, Units, # Risks, Nr. Of units, # Units, # of buildings, *# of Bldgs, # of Units  
- **Square Footage**: Square Footage, Actual Sq Ft Used, *Square Footage, Total SqFt, $/Sqft  
- **Building Value**: Building Limit, Real property value, Building coverage, %Building coverage%, %Building limit%, Building value, %Building value%, *Real Property Value ($), Real Property Value ($), Buildings - Replacement Value  
- **Contents Value**: Contents value, %Content coverage%, Content coverage, %Personal property value%, Personal property value, Personal Property Value ($), Contents  
- **BI Value**: Business Interruption value, Business Interruption, %Business Interruption%, BI, %BI%, Business Income, %Business Income%, Time value, %Time value%, BI/EE  
- **Rental Value**: Rental Value, Rental Income, Annual Rental Income, BI/Rental Income($), Rental, %Rental Income%, Loss of rent, %Loss of rent%  
- **M&E Value**: M&E, %M&E%, Machinery & Equipment, %Machinery & Equipment%, M&E Value, M&E (Complete M&E Tech Summary Sheet)  
- **Other Value**: Other Value, %Other value%, Other limit, %Other limit%, Other coverage, %Other coverage, Other Value $ (outdoor prop & Eqpt must be sch'd), Other  
- **TIV**: TIV, %TIV%, Total Insured Value, Total TIV, Total Insurable Value(TIV), Total, Total Insurable Value, Total Insurable Values, *Total TIV, Formula Expiring Total TIV, Total Covered Value  
- **Property type**: Property type, *Property type  
- **SOV Occupancy Description**: Building Occupancy, SOV Occupancy Description, *SOV Occupancy Description, Occupancy Description, Occupancy type, *Occupancy, Occupancy, Description, Occupancy Details, WKFC Detailed Occupancy, description/occupancy, Occ, OWNER/ENTITY/GENERAL PARTNER\nAdd'l Named Insured, *Occupancy Description  
- **Percentage Occupied**: *Percentage Occupied, % Occupied, Percentage Occupied  
- **Year Built**: *Year built, Year built, Construction year, Original year built, *Orig Year Built, Orig Year Built  
- **Year Updated**: Year Updated, *Year Updated, Yr Bldg upgraded - Major Exterior Update, Yr Bldg updated (Mand if >25 yrs), Modified year, %Building upgraded%, Effective year built, Year Renovated  
- **Roof replacement year**: *Roof replacement year, Roof replacement year, Roof rebuild year, Dates of updates to roof, hvac, wiring, plumbing, *Yr. Roof covering last repl, Year Last Roof  
- **SOV Construction Description**: *SOV Construction Description, Constr., RMS Construction Class, Construction, Construction Description, CONSTRUCTION DESCRIPTION OF (...), Construction Description (provide further details on construction features), Construction Type, ISO Construction Type, *ISO Const, Amrisc construction  
- **Roof Condition**: Roof condition, *Roof condition, Roof quality, Roof Type, Roof Construction  
- **Ownership Entity**: Ownership Entiry, *Ownership Entiry, Entity, Ownership  

### Markdown-formatted Table

{markdown_chunk}

---
### Your Tasks
 
### 1. **Identify the header row(s)**:
   - Analyze the dataset to find rows that likely represent column names, based on known header name variations.
   - If a single row clearly contains the column headers, return that row as the `header_row`.
   - If the header spans multiple rows (e.g., subheaders, annotations, or hierarchical labels), return a tuple of row indices that together form the complete header.
   - If no confident match is found, return `"header_row": null` and provide a detailed explanation in the `additional_info`.

---

### Output Clarification:

- **header_row**:  
   - If the header is determined to span multiple rows, return a tuple of the row indices.
   - If only one row serves as the header, return the index of that row.
   - If no header row can be identified, return `null` and include an explanation in the `additional_info`.
---

### Example Output:

```json
{{
  "header_row": <int | (int, int) | null>,
  "metadata": {{
    "additional_info": "<any notes, explaination or fallback logic>"
  }}
}}
    
"""
    
    result = get_completion(prompt = user_prompt, system_prompt = system_prompt)
    
    return json.loads(result), preview_df

In [7]:
file_path =r"data/97328286_Marketing SOV - Copy.xls" # Repace with your file
sheet_num = 0 # replace with sheet num of sov in the file 

In [8]:
# Select the 4th sheet (index 3) from the Excel file
sheet_name = pd.ExcelFile(file_path).sheet_names[sheet_num]
print(f"Sheet Name: {sheet_name}")
print("-" * 80)

# Extract header row number and dataframe for the selected sheet
header_row_num_result, p_df = extract_header_row_number_only(file_path, sheet_name=sheet_name)
print("\nHeader Row Detection Result:")
print("-" * 80)
header_row_num_result

Sheet Name: SOV-APP
--------------------------------------------------------------------------------

Header Row Detection Result:
--------------------------------------------------------------------------------


{'header_row': 11,
 'metadata': {'additional_info': "Row 11 contains a high density of matches to the standard header names and their variants, such as '* Bldg No.', 'Location Name', 'AddressNum', '*Street Address', '*City', '*State Code', '*Zip', 'County', '*# of Bldgs', '*ISO Const', 'Construction Description', '*# of Stories', '*Orig Year Built', '*Real Property Value ($)', 'Personal Property Value ($)', 'Other Value $ (outdoor prop & Eqpt must be sch'd)', 'BI/Rental Income ($)', '*Total TIV', '*Occupancy', '*# of Units', '*Square Footage', and many more. The structure and content of this row are consistent with a typical header row in an SOV (Statement of Values) or property schedule. The rows above (0-10) contain either metadata, instructions, or are empty, and the rows below (12+) contain data. There is no evidence of a multi-row header structure."}}

## 2.2 Function to Get the Actual Header using Header Row Number - working fine 

In [9]:
def get_dataframe_header_from_excel(preview_df, header_row):
    try:
        if header_row is None:
            raise ValueError("Header row is not provided.")
        
        if isinstance(header_row, tuple):
            start_row, end_row = header_row
            header = preview_df.iloc[start_row:end_row + 1]
            header = header.astype(str).apply(lambda x: ' '.join(x), axis=0)
        else:
            header = preview_df.iloc[header_row].astype(str)

        return header

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

header_row = get_dataframe_header_from_excel(p_df, header_row=header_row_num_result['header_row'])
print("-" * 80)
print("Extracted Header Row:")
print("-" * 80)
header_row.to_list()

### 2.3 Function to Map the Raw headers to Standardized header (using LLM)- Problem

In [10]:
def map_headers(header_row):
    """This function takes a header row and maps raw headers to standardized names
    based on a predefined list of possible variations.
    Parameters:
    - header_row: The row containing the raw header names from the Excel sheet.
    Returns:
    - A JSON result containing the header mappings and metadata."""

    system_prompt = """
You are a helpful assistant that standardizes Excel headers.
Your task is to match raw headers to known standardized names based on a list of possible variations. If there’s no direct match, use semantic similarity.
Only focus on headers — do not interpret or extract data rows.
    """
    user_prompt = f"""
You are given the raw headers extracted from an Excel sheet:
{header_row}
---
### Standard Header Names & Variants

Match each raw header to a standard name based on the keyword variants listed below. These are some of the acceptable variations you can match against (case-insensitive, ignore punctuation and spacing):

- **Location Name**: Location Name, Name, Description, *Location, *Location Name, *Description  
- **Location Number**: Location Number, Location #, Loc No, Location  
- **Bldg No**: *Bldg No., *Bldg No, Bldg No., Building Number, Building #, Bldg. #  
- **Property ID**: Property ID, PID, *Property ID  
- **House Number**: House number, Address number, AddressNum, Address  
- **Street Address**: Street Address, address, *Street Address  
- **City**: City, *City  
- **Zip Code**: Zip, Zip code, *Zip code, Pincode, Pin, *Zip  
- **State**: State, *State Code, State Code, ST  
- **County**: County  
- **Country**: Country, *Country  
- **Number of Units**: Number of Units, Nr. of risks, Units, # Risks, Nr. Of units, # Units, # of buildings, *# of Bldgs, # of Units  
- **Square Footage**: Square Footage, Actual Sq Ft Used, *Square Footage, Total SqFt, $/Sqft  
- **Building Value**: Building Limit, Real property value, Building coverage, %Building coverage%, %Building limit%, Building value, %Building value%, *Real Property Value ($), Real Property Value ($), Buildings - Replacement Value  
- **Contents Value**: Contents value, %Content coverage%, Content coverage, %Personal property value%, Personal property value, Personal Property Value ($), Contents  
- **BI Value**: Business Interruption value, Business Interruption, %Business Interruption%, BI, %BI%, Business Income, %Business Income%, Time value, %Time value%, BI/EE  
- **Rental Value**: Rental Value, Rental Income, Annual Rental Income, BI/Rental Income($), Rental, %Rental Income%, Loss of rent, %Loss of rent%  
- **M&E Value**: M&E, %M&E%, Machinery & Equipment, %Machinery & Equipment%, M&E Value, M&E (Complete M&E Tech Summary Sheet)  
- **Other Value**: Other Value, %Other value%, Other limit, %Other limit%, Other coverage, %Other coverage, Other Value $ (outdoor prop & Eqpt must be sch'd), Other  
- **TIV**: TIV, %TIV%, Total Insured Value, Total TIV, Total Insurable Value(TIV), Total, Total Insurable Value, Total Insurable Values, *Total TIV, Formula Expiring Total TIV, Total Covered Value  
- **Property type**: Property type, *Property type  
- **SOV Occupancy Description**: Building Occupancy, SOV Occupancy Description, *SOV Occupancy Description, Occupancy Description, Occupancy type, *Occupancy, Occupancy, Description, Occupancy Details, WKFC Detailed Occupancy, description/occupancy, Occ, OWNER/ENTITY/GENERAL PARTNER\nAdd'l Named Insured, *Occupancy Description  
- **Percentage Occupied**: *Percentage Occupied, % Occupied, Percentage Occupied  
- **Year Built**: *Year built, Year built, Construction year, Original year built, *Orig Year Built, Orig Year Built  
- **Year Updated**: Year Updated, *Year Updated, Yr Bldg upgraded - Major Exterior Update, Yr Bldg updated (Mand if >25 yrs), Modified year, %Building upgraded%, Effective year built, Year Renovated  
- **Roof replacement year**: *Roof replacement year, Roof replacement year, Roof rebuild year, Dates of updates to roof, hvac, wiring, plumbing, *Yr. Roof covering last repl, Year Last Roof  
- **SOV Construction Description**: *SOV Construction Description, Constr., RMS Construction Class, Construction, Construction Description, CONSTRUCTION DESCRIPTION OF (...), Construction Description (provide further details on construction features), Construction Type, ISO Construction Type, *ISO Const, Amrisc construction  
- **Roof Condition**: Roof condition, *Roof condition, Roof quality, Roof Type, Roof Construction  
- **Ownership Entity**: Ownership Entiry, *Ownership Entiry, Entity, Ownership  

---
### Your Tasks

1. **Map raw headers to standardized names**:
   - Use the variants above to determine which standardized name each raw header maps to.
   - If multiple raw headers map to the same standard name, keep only the most descriptive.
   - If no match (direct or semantic), skip that header.
   - Do not change or clean the raw header text — return it exactly as it appears.
 
2. **Extract metadata**:
   - Identify and return values for `carrier`, `insured`, and `broker` if they appear anywhere.
----

### Output Format

```json
{{
  "header_mapping": {{
    "Standard Header 1": "Raw Header 1",
    "Standard Header 2": "Raw Header 2"
  }},
  "metadata": {{
    "carrier": "<value>",
    "insured": "<value>",
    "broker": "<value>",
    "additional_info": "<any notes or fallback logic>"
  }}
}}```
"""

    result = get_completion(prompt = user_prompt, system_prompt = system_prompt)
    
    return json.loads(result)

### Test for functions

In [11]:
def extract_headers(file_path, sheet_num=0):
    sheet_name = pd.ExcelFile(file_path).sheet_names[sheet_num]
    print(f"Sheet Name: {sheet_name}")
    header_row_num_result, p_df = extract_header_row_number_only(file_path, sheet_name=sheet_name)
    header_row = get_dataframe_header_from_excel(p_df, header_row=header_row_num_result['header_row'])
    mapped_results = map_headers(header_row.to_list())

    return {
        'header_row_number': header_row_num_result['header_row'],
        'extracted_headers': header_row.to_list(),
        'header_mapping': mapped_results['header_mapping']
    }

In [12]:
file_path =r"data/97328286_Marketing SOV - Copy.xls" # Repace with your file
sheet_num = 0 # replace with sheet num of sov in the file 

result = extract_headers(file_path, sheet_num)
result

Sheet Name: SOV-APP


{'header_row_number': 11,
 'extracted_headers': ['* Bldg No.',
  'Location Name',
  'AddressNum',
  '*Street Address',
  '*City',
  '*State Code',
  '*Zip',
  'County',
  'Is Prop within 1000 ft of saltwater',
  '*# of Bldgs',
  '*ISO Const',
  'Construction Description',
  '*# of Stories',
  '*Orig Year Built',
  'nan',
  '*Real Property Value ($)',
  'Personal Property Value ($)',
  "Other Value $ (outdoor prop & Eqpt must be sch'd)",
  'BI/Rental Income ($)',
  '*Total TIV',
  '*Occupancy',
  '*# of Units',
  '*Square Footage',
  'Percent Sprinklered',
  'ISO Prot Class',
  'Group1-6',
  'Sprinklered (Y/N)',
  'Yr Bldg updated (Mand if >25 yrs)',
  'Flood Zone',
  '*Basement',
  '*Building Foundation',
  '*Construction Quality',
  '**Roof Framing/Structure',
  '**Shape of roof',
  '**Type of Roof Covering',
  '*Roof strapped (if ISO 1 OR 2)',
  'Building Maintenance',
  '*Yr. Roof covering last repl',
  '**Exterior Cladding',
  '*EIFS? - Coverage warrants no EIFS',
  'Wind Pool Elig